# Within Sample Prediction

Our ultimate goal is to create a model that can be used to predict events that have not yet happened. Before we do that, however, we ought to check first whether our model fits the data from the past. It's not impossible that a model which fits the historical record badly can be a good forecasting model, (especially if the world is changing dramatically), but under most circumstances, we should expect a model to work going backwards as well as forwards.

As outlined in the previous session, we're going to use the Transfermarkt (TM) data for forecasting, and we've already seen that the TM data for 2011-2018 fits the wage data from the financial statements well. So now let's see how well the TM data performs when explaining the variation of game results in the Premier League.

What should we expect? Anyone hoping for anything close to a 100% fit should give up now. If any sport were that predictable, then it would be no fun to watch in the first place. Even if some aspects of the game seem highly predictable, surprises happen all the time at the level of the individual game.

The accuracy of bookmakers is a much better standard to consider. Bookmakers make money by setting the odds correctly- if they get them badly wrong then punters can bet against them and make easy money, which will also make the bookmaker bankrupt. The only bookmakers we see are the ones that have not gone bankrupt, so it is reasonable to think that they are getting odds right on average. What does that mean? Odds are probabilities, and a bookmaker makes money by offering probabilities that are (more or less) correct on average. So if we use our model to generate probabilities, we can compare them with bookmaker probabilities, and see how close our model gets.

Reality check: if we could build a model much better than the bookmakers, then we wouldn't be academics, we'd be multi-millionaire gamblers living in the Bahamas. Some people claim to have built a model that can systematically beat the bookmakers, but such people seldom offer to show you the model. There's an entire business built around *telling* people you've built a model that beats the bookmakers, and then scamming these people for money. 

Think of it this way- for any game there are "true" probabilities, and the business of bookmakers is to get as close to these true probabilities as possible. In general, the bookmaker odds are close to the true probabilities, so if we have built a good model it can get close to the bookmaker probabilities.

What we will show in this notebook is that, perhaps surprisingly, it's not very difficult to get close.

There are three steps in this process:

    Step 1: Calculate the accuracy of betting odds in predicting outcomes.
    Step 2: Calculate predictions based on a model using the TMvalues.
    Step 3: Compare betting odds and our model's predictions using these metrics: (a) correct outcome (b) Brier score

But first, as always we set up our data.

In [1]:
# Enable full screen width

from IPython.display import display, HTML

display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }
</style>
"""))

In [2]:
# import packages

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf

We will use two datasets. TMdat we used in the last session, and now we also load a df of game results for the Premier League for seasons 2010/11 to 2018/19:

In [3]:
TMdat = pd.read_excel("../../Data/Week 3/EPL TM values 2010-11 to 2019-20.xlsx")
gbygdat = pd.read_excel("../../Data/Week 3/EPL game results 2011-19 with odds.xlsx")

In [4]:
gbygdat

,seasonyrend,home_team,away_team,FTHG,FTAG,FTR,B365H,B365D,B365A
0,2011,Aston Villa,Arsenal,2,4,A,3.40,3.30,2.20
1,2011,Birmingham City,Arsenal,0,3,A,5.50,3.60,1.67
2,2011,Blackburn Rovers,Arsenal,1,2,A,6.00,4.00,1.57
3,2011,Blackpool,Arsenal,1,3,A,8.50,5.00,1.36
4,2011,Bolton Wanderers,Arsenal,2,1,H,4.50,3.80,1.75
...,...,...,...,...,...,...,...,...,...
3415,2019,Wolverhampton Wanderers,Newcastle United,1,1,D,1.70,3.75,5.75
3416,2019,Wolverhampton Wanderers,Southampton,2,0,H,1.80,3.60,5.25
3417,2019,Wolverhampton Wanderers,Tottenham Hotspur,2,3,A,3.10,3.40,2.45
3418,2019,Wolverhampton Wanderers,Watford,0,2,A,1.75,3.75,5.25


Looking at a gbygdat, you can see that it lists 3420 games. It identifies the season, home team and away team. FTHG is the total goals scored in the game by the home team, FTAG is the total goals scored by the away team. This is enough to tell us the result, but this is also provided in next column: H = home win, A = away win, D = draw (tie). 

In addition, we have the decimal odds on the three possible outcomes for each game, as offered before the game by the bookmaker Bet365. These odds can be obtained for free, along with the odds of many other bookmakers, for many different leagues, at this website: https://www.football-data.co.uk/. 

Although we only use one bookmaker as our benchmark, the odds set by all the bookmakers are typically very close together, rather in the same way that the exchange rate for currency trades offered by a broker is very close to that offered by all other brokers. If that were not the case, there would be an easy opportunity to make money through arbitrage, which means buying from one broker and selling to another, making a profit without taking any risk. The opportunity to make arbitrage profits in bookmaking is not unknown, but is very rare.

## Step 1: Calculate the accuracy of betting odds in predicting outcomes

We want to convert the bookmaker odds to probabilities and see how accurate they are. The odds are in decimal format, and therefore the outcome probability equals 1/(decimal odds).  However, if you make this calculation and sum the three possibilities the total is greater than one.  This is called the 'overround', or the 'vig' - and represents the profit of the bookmaker. To calculate the implied probability from the betting odds you have to divide by the sum of the three numbers, so that your final probabilities do add up to 1 (100%).

We calculate these probabilities below:

In [5]:
gbygdat['B365HPr']= 1/(gbygdat['B365H'])/(1/(gbygdat['B365H'])+ 1/(gbygdat['B365D'])+ 1/(gbygdat['B365A']))
gbygdat['B365DPr']= 1/(gbygdat['B365D'])/(1/(gbygdat['B365H'])+ 1/(gbygdat['B365D'])+ 1/(gbygdat['B365A']))
gbygdat['B365APr']= 1/(gbygdat['B365A'])/(1/(gbygdat['B365H'])+ 1/(gbygdat['B365D'])+ 1/(gbygdat['B365A']))
gbygdat

,seasonyrend,home_team,away_team,FTHG,FTAG,FTR,B365H,B365D,B365A,B365HPr,B365DPr,B365APr
0,2011,Aston Villa,Arsenal,2,4,A,3.40,3.30,2.20,0.279661,0.288136,0.432203
1,2011,Birmingham City,Arsenal,0,3,A,5.50,3.60,1.67,0.171786,0.262451,0.565763
2,2011,Blackburn Rovers,Arsenal,1,2,A,6.00,4.00,1.57,0.158186,0.237280,0.604534
3,2011,Blackpool,Arsenal,1,3,A,8.50,5.00,1.36,0.111732,0.189944,0.698324
4,2011,Bolton Wanderers,Arsenal,2,1,H,4.50,3.80,1.75,0.210277,0.249012,0.540711
...,...,...,...,...,...,...,...,...,...,...,...,...
3415,2019,Wolverhampton Wanderers,Newcastle United,1,1,D,1.70,3.75,5.75,0.571760,0.259198,0.169042
3416,2019,Wolverhampton Wanderers,Southampton,2,0,H,1.80,3.60,5.25,0.542636,0.271318,0.186047
3417,2019,Wolverhampton Wanderers,Tottenham Hotspur,2,3,A,3.10,3.40,2.45,0.314755,0.286983,0.398262
3418,2019,Wolverhampton Wanderers,Watford,0,2,A,1.75,3.75,5.25,0.555556,0.259259,0.185185


These are probabilities for each possible outcome. We can also say that the predicted outcome of any game is the one with the highest probability. We can use 'np.where' to identify the implied result from the probabilities. For example, if the probability of home win is greater than the probability of a draw AND the probability of home win is greater than the probability of an away win, then home win is the predicted outcome (H). 

Note: we could just as easily have generated this variable from the decimal odds, since the outcome with the lowest decimal odds has the highest probability.

In [6]:
gbygdat['B365res']= np.where((gbygdat['B365HPr']>gbygdat['B365DPr']) & (gbygdat['B365HPr']>gbygdat['B365APr']),'H',\
                            np.where((gbygdat['B365DPr']>gbygdat['B365HPr']) & (gbygdat['B365DPr']>gbygdat['B365APr']),'D','A'))
gbygdat

,seasonyrend,home_team,away_team,FTHG,FTAG,FTR,B365H,B365D,B365A,B365HPr,B365DPr,B365APr,B365res
0,2011,Aston Villa,Arsenal,2,4,A,3.40,3.30,2.20,0.279661,0.288136,0.432203,A
1,2011,Birmingham City,Arsenal,0,3,A,5.50,3.60,1.67,0.171786,0.262451,0.565763,A
2,2011,Blackburn Rovers,Arsenal,1,2,A,6.00,4.00,1.57,0.158186,0.237280,0.604534,A
3,2011,Blackpool,Arsenal,1,3,A,8.50,5.00,1.36,0.111732,0.189944,0.698324,A
4,2011,Bolton Wanderers,Arsenal,2,1,H,4.50,3.80,1.75,0.210277,0.249012,0.540711,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3415,2019,Wolverhampton Wanderers,Newcastle United,1,1,D,1.70,3.75,5.75,0.571760,0.259198,0.169042,H
3416,2019,Wolverhampton Wanderers,Southampton,2,0,H,1.80,3.60,5.25,0.542636,0.271318,0.186047,H
3417,2019,Wolverhampton Wanderers,Tottenham Hotspur,2,3,A,3.10,3.40,2.45,0.314755,0.286983,0.398262,A
3418,2019,Wolverhampton Wanderers,Watford,0,2,A,1.75,3.75,5.25,0.555556,0.259259,0.185185,H


What percentage of the time was the bookmaker's prediction correct? We can create a variable which equals one when the actual result (FTR) equals the bookmaker's prediction (B365res), and zero otherwise:

In [7]:
gbygdat['B365true']= np.where(gbygdat['B365res'] == gbygdat['FTR'],1,0)
gbygdat.describe(include='all')

,seasonyrend,home_team,away_team,FTHG,FTAG,FTR,B365H,B365D,B365A,B365HPr,B365DPr,B365APr,B365res,B365true
count,3420.000000,3420,3420,3420.000000,3420.000000,3420,3420.000000,3420.000000,3420.000000,3420.000000,3420.000000,3420.000000,3420,3420.000000
unique,NaN,35,35,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN
top,NaN,Manchester City,Manchester City,NaN,NaN,H,NaN,NaN,NaN,NaN,NaN,NaN,H,NaN
freq,NaN,171,171,NaN,NaN,1565,NaN,NaN,NaN,NaN,NaN,NaN,2373,NaN
mean,2015.000000,NaN,NaN,1.556433,1.191813,NaN,2.830766,4.123731,5.037944,0.450754,0.248720,0.300526,NaN,0.544444
std,2.582366,NaN,NaN,1.304716,1.164475,NaN,2.039274,1.279006,4.313887,0.194380,0.049793,0.173229,NaN,0.498094
min,2011.000000,NaN,NaN,0.000000,0.000000,NaN,1.060000,3.000000,1.120000,0.042139,0.057020,0.023781,NaN,0.000000
25%,2013.000000,NaN,NaN,1.000000,0.000000,NaN,1.660000,3.400000,2.400000,0.314755,0.224913,0.169579,NaN,0.000000
50%,2015.000000,NaN,NaN,1.000000,1.000000,NaN,2.200000,3.600000,3.500000,0.441776,0.264999,0.271396,NaN,1.000000
75%,2017.000000,NaN,NaN,2.000000,2.000000,NaN,3.100000,4.330000,5.750000,0.585502,0.286816,0.397351,NaN,1.000000


The percentage of correct predictions is just the mean of this variable, B365true:

In [8]:
gbygdat['B365true'].mean()

0.5444444444444444

It's also interesting to compare the cases where the betting odds were "correct" (in the sense of attaching the highest probability to the actual outcome), to those were they were not. We can do this using the .groupby command for the variable 'B365true'. You can see below that when the betting odds were correct, on average the home team scored two goals to the away team's one, and that the home team had a higher probability of winning according to the betting odds. Where the bookmaker failed, the odds were more balanced and the outcome in terms of goals actually favoured the away team on average. This suggests that the bookmakers are better at correctly home wins when the home team is much stronger than the away team, and less successful at predicting close games where the away team often wins.

In [9]:
gbygdat.groupby('B365true').mean()

,seasonyrend,FTHG,FTAG,B365H,B365D,B365A,B365HPr,B365DPr,B365APr
B365true,,,,,,,,,
0,2014.897304,1.082157,1.296534,2.787651,3.776200,4.080969,0.420245,0.263857,0.315898
1,2015.085929,1.953276,1.104189,2.866842,4.414522,5.838679,0.476282,0.236054,0.287664


So, a 54% success rate represents a benchmark against which we can compare the result of our own model. Note that 54% is not necessarily that great. Suppose you picked at random, how many results would you get right? If you guessed on the basis that each outcome was equally likely, you'd be right about one-third of the time. So the range of reasonable success rates lies somewhere in the range of 33% (random) and 54% (the bookmakers).

Attaching the highest probability to the actual outcome is one measure of success, but as we discussed when looking at the betting odds, the Brier Score is a better measure, since it measures the closeness of predicted probabilities to the outcome.

So, let's now derive the Brier Score for the Bet365 data. Recall that the Brier score is defined as the sum of squared differences between the value of each actual outcome (which is one for the actual result, and zero for the two outcomes that were not realized) and the predicted probability of that outcome, all divided by the number of events predicted (which in our case is 3420 games).

First, we define a variable for each possible outcome, with the value 1 if this was the actual outcome and zero otherwise:

In [10]:
gbygdat['Houtcome']= np.where(gbygdat['FTR']=='H',1,0)
gbygdat['Doutcome']= np.where(gbygdat['FTR']=='D',1,0)
gbygdat['Aoutcome']= np.where(gbygdat['FTR']=='A',1,0)
gbygdat

,seasonyrend,home_team,away_team,FTHG,FTAG,FTR,B365H,B365D,B365A,B365HPr,B365DPr,B365APr,B365res,B365true,Houtcome,Doutcome,Aoutcome
0,2011,Aston Villa,Arsenal,2,4,A,3.40,3.30,2.20,0.279661,0.288136,0.432203,A,1,0,0,1
1,2011,Birmingham City,Arsenal,0,3,A,5.50,3.60,1.67,0.171786,0.262451,0.565763,A,1,0,0,1
2,2011,Blackburn Rovers,Arsenal,1,2,A,6.00,4.00,1.57,0.158186,0.237280,0.604534,A,1,0,0,1
3,2011,Blackpool,Arsenal,1,3,A,8.50,5.00,1.36,0.111732,0.189944,0.698324,A,1,0,0,1
4,2011,Bolton Wanderers,Arsenal,2,1,H,4.50,3.80,1.75,0.210277,0.249012,0.540711,A,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3415,2019,Wolverhampton Wanderers,Newcastle United,1,1,D,1.70,3.75,5.75,0.571760,0.259198,0.169042,H,0,0,1,0
3416,2019,Wolverhampton Wanderers,Southampton,2,0,H,1.80,3.60,5.25,0.542636,0.271318,0.186047,H,1,1,0,0
3417,2019,Wolverhampton Wanderers,Tottenham Hotspur,2,3,A,3.10,3.40,2.45,0.314755,0.286983,0.398262,A,1,0,0,1
3418,2019,Wolverhampton Wanderers,Watford,0,2,A,1.75,3.75,5.25,0.555556,0.259259,0.185185,H,0,0,0,1


Now calculate the Brier Score as the sum of the squared differences divided by the number of events:

In [11]:
BrierB365 = ((gbygdat['B365HPr'] - gbygdat['Houtcome'])**2 +(gbygdat['B365DPr'] - gbygdat['Doutcome'])**2 +\
             (gbygdat['B365APr'] - gbygdat['Aoutcome'])**2).sum()/3420 
BrierB365

0.568726835315728

This value has little meaning in its own right. Its value lies in providing a benchmark for the predictions which we will now generate using a model based on TM values.

## Step 2: Calculate predictions using the TM values

### The Model

We are going to use a very simple model: the probability that a team wins a game depends on (a) home advantage and (b) the ratio of the quality of the teams, as measured by the TM value. 

This model is simply the game level version of what we found out in Course 1, namely that teams whose player spending is higher are more likely to win. 

We need to estimate our model in order to identify exactly how much the probability of winning is affected by home advantage and relative team quality. To do this we are going to use a regression model, estimate the best fit between our explanatory variables (home advantage and TM ratio) and the dependent variable, which is the outcome of the game (H, D, A).

As we saw in the beginning of this course, the appropriate regression model for this situation is the ordered logit model, which estimates the probability of each possible outcome based on the data. These estimates, then, will be our model's prediction, and we can compare the accuracy of these estimates to the estimates implied by the betting odds above.

In our estimation, we will take the perspective of the home team. That means when the TM ratio will have the home team TM value in the numerator and the away team TM value in the denominator. This also means that the estimated intercept will be an indicator of home team advantage. In fact, if we use the logarithm of the TM ratio, when the two sides have equal spending the value will be zero (log(1) = 0) and hence the intercept can be interpreted as pure home advantage, independent of player spending.




### Organizing the data

Before running the regression we must first organize the data. We need to merge the files with the betting odds and the TM values (we will add the TM values to the gbygdat df). To do this we need to create a matching variable. If we create a variable which is the name of each club and the year value (e.g. Arsenal2011) then we can use this as the basis for merging the TM values in gbygdat.

This is as simple as just adding the two variable names together, although remembering that while the team names are already strings, the year values are integers, and to combine the names we need to treat integer values as strings.

We create one variable for the home team - 'hteamid' - and one for the away team - 'ateamid'.

In [12]:
# first we create the merge id in gbygdat

gbygdat['hteamid'] = gbygdat['home_team']+gbygdat['seasonyrend'].map(str)
gbygdat['ateamid'] = gbygdat['away_team']+gbygdat['seasonyrend'].map(str)

gbygdat

,seasonyrend,home_team,away_team,FTHG,FTAG,FTR,B365H,B365D,B365A,B365HPr,B365DPr,B365APr,B365res,B365true,Houtcome,Doutcome,Aoutcome,hteamid,ateamid
0,2011,Aston Villa,Arsenal,2,4,A,3.40,3.30,2.20,0.279661,0.288136,0.432203,A,1,0,0,1,Aston Villa2011,Arsenal2011
1,2011,Birmingham City,Arsenal,0,3,A,5.50,3.60,1.67,0.171786,0.262451,0.565763,A,1,0,0,1,Birmingham City2011,Arsenal2011
2,2011,Blackburn Rovers,Arsenal,1,2,A,6.00,4.00,1.57,0.158186,0.237280,0.604534,A,1,0,0,1,Blackburn Rovers2011,Arsenal2011
3,2011,Blackpool,Arsenal,1,3,A,8.50,5.00,1.36,0.111732,0.189944,0.698324,A,1,0,0,1,Blackpool2011,Arsenal2011
4,2011,Bolton Wanderers,Arsenal,2,1,H,4.50,3.80,1.75,0.210277,0.249012,0.540711,A,0,1,0,0,Bolton Wanderers2011,Arsenal2011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3415,2019,Wolverhampton Wanderers,Newcastle United,1,1,D,1.70,3.75,5.75,0.571760,0.259198,0.169042,H,0,0,1,0,Wolverhampton Wanderers2019,Newcastle United2019
3416,2019,Wolverhampton Wanderers,Southampton,2,0,H,1.80,3.60,5.25,0.542636,0.271318,0.186047,H,1,1,0,0,Wolverhampton Wanderers2019,Southampton2019
3417,2019,Wolverhampton Wanderers,Tottenham Hotspur,2,3,A,3.10,3.40,2.45,0.314755,0.286983,0.398262,A,1,0,0,1,Wolverhampton Wanderers2019,Tottenham Hotspur2019
3418,2019,Wolverhampton Wanderers,Watford,0,2,A,1.75,3.75,5.25,0.555556,0.259259,0.185185,H,0,0,0,1,Wolverhampton Wanderers2019,Watford2019


Now we do the same thing in TMdat. For the purposes of matching, we need to match with the home team and the away team in the gbygdat, but in our TM file, we just have the TM value for each club. For each game, the club might be the home team or the away team. So we simply create two versions of the same variable and give them different names- hteamid and ateamid, which we can then use for the merge:

In [13]:
TMdat['hteamid'] = TMdat['Club']+TMdat['seasonyrend'].astype(str)
TMdat['ateamid'] = TMdat['Club']+TMdat['seasonyrend'].astype(str)

TMdat

,seasonyrbegin,seasonyrend,Club,TMValue,hteamid,ateamid
0,2010,2011,Arsenal,272.70,Arsenal2011,Arsenal2011
1,2010,2011,Aston Villa,127.94,Aston Villa2011,Aston Villa2011
2,2010,2011,Birmingham City,68.00,Birmingham City2011,Birmingham City2011
3,2010,2011,Blackburn Rovers,65.93,Blackburn Rovers2011,Blackburn Rovers2011
4,2010,2011,Blackpool,24.37,Blackpool2011,Blackpool2011
...,...,...,...,...,...,...
195,2019,2020,Southampton,209.70,Southampton2020,Southampton2020
196,2019,2020,Tottenham Hotspur,881.55,Tottenham Hotspur2020,Tottenham Hotspur2020
197,2019,2020,Watford,214.52,Watford2020,Watford2020
198,2019,2020,West Ham United,299.03,West Ham United2020,West Ham United2020


So now we can do two merges, one for hteamid and the other for ateamid. We are merging just the TMvalue column into the gbygdat df, using each of the teamids.

We merge on hteamid first. We will rename the TM value column hTM to denote that it is the value matched for the home team:

In [14]:
gbygdat = pd.merge(gbygdat,TMdat[['hteamid','TMValue']], on= 'hteamid', how = 'left').rename(columns={'TMValue':'hTM'})
gbygdat

,seasonyrend,home_team,away_team,FTHG,FTAG,FTR,B365H,B365D,B365A,B365HPr,B365DPr,B365APr,B365res,B365true,Houtcome,Doutcome,Aoutcome,hteamid,ateamid,hTM
0,2011,Aston Villa,Arsenal,2,4,A,3.40,3.30,2.20,0.279661,0.288136,0.432203,A,1,0,0,1,Aston Villa2011,Arsenal2011,127.94
1,2011,Birmingham City,Arsenal,0,3,A,5.50,3.60,1.67,0.171786,0.262451,0.565763,A,1,0,0,1,Birmingham City2011,Arsenal2011,68.00
2,2011,Blackburn Rovers,Arsenal,1,2,A,6.00,4.00,1.57,0.158186,0.237280,0.604534,A,1,0,0,1,Blackburn Rovers2011,Arsenal2011,65.93
3,2011,Blackpool,Arsenal,1,3,A,8.50,5.00,1.36,0.111732,0.189944,0.698324,A,1,0,0,1,Blackpool2011,Arsenal2011,24.37
4,2011,Bolton Wanderers,Arsenal,2,1,H,4.50,3.80,1.75,0.210277,0.249012,0.540711,A,0,1,0,0,Bolton Wanderers2011,Arsenal2011,74.93
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3415,2019,Wolverhampton Wanderers,Newcastle United,1,1,D,1.70,3.75,5.75,0.571760,0.259198,0.169042,H,0,0,1,0,Wolverhampton Wanderers2019,Newcastle United2019,178.02
3416,2019,Wolverhampton Wanderers,Southampton,2,0,H,1.80,3.60,5.25,0.542636,0.271318,0.186047,H,1,1,0,0,Wolverhampton Wanderers2019,Southampton2019,178.02
3417,2019,Wolverhampton Wanderers,Tottenham Hotspur,2,3,A,3.10,3.40,2.45,0.314755,0.286983,0.398262,A,1,0,0,1,Wolverhampton Wanderers2019,Tottenham Hotspur2019,178.02
3418,2019,Wolverhampton Wanderers,Watford,0,2,A,1.75,3.75,5.25,0.555556,0.259259,0.185185,H,0,0,0,1,Wolverhampton Wanderers2019,Watford2019,178.02


Now we merge on ateamid. We will rename the TM value column aTM to denote that it is the value matched for the away team:

In [15]:
gbygdat = pd.merge(gbygdat,TMdat[['ateamid','TMValue']], on= 'ateamid', how = 'left').rename(columns={'TMValue':'aTM'})
gbygdat

,seasonyrend,home_team,away_team,FTHG,FTAG,FTR,B365H,B365D,B365A,B365HPr,...,B365APr,B365res,B365true,Houtcome,Doutcome,Aoutcome,hteamid,ateamid,hTM,aTM
0,2011,Aston Villa,Arsenal,2,4,A,3.40,3.30,2.20,0.279661,...,0.432203,A,1,0,0,1,Aston Villa2011,Arsenal2011,127.94,272.70
1,2011,Birmingham City,Arsenal,0,3,A,5.50,3.60,1.67,0.171786,...,0.565763,A,1,0,0,1,Birmingham City2011,Arsenal2011,68.00,272.70
2,2011,Blackburn Rovers,Arsenal,1,2,A,6.00,4.00,1.57,0.158186,...,0.604534,A,1,0,0,1,Blackburn Rovers2011,Arsenal2011,65.93,272.70
3,2011,Blackpool,Arsenal,1,3,A,8.50,5.00,1.36,0.111732,...,0.698324,A,1,0,0,1,Blackpool2011,Arsenal2011,24.37,272.70
4,2011,Bolton Wanderers,Arsenal,2,1,H,4.50,3.80,1.75,0.210277,...,0.540711,A,0,1,0,0,Bolton Wanderers2011,Arsenal2011,74.93,272.70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3415,2019,Wolverhampton Wanderers,Newcastle United,1,1,D,1.70,3.75,5.75,0.571760,...,0.169042,H,0,0,1,0,Wolverhampton Wanderers2019,Newcastle United2019,178.02,159.03
3416,2019,Wolverhampton Wanderers,Southampton,2,0,H,1.80,3.60,5.25,0.542636,...,0.186047,H,1,1,0,0,Wolverhampton Wanderers2019,Southampton2019,178.02,239.72
3417,2019,Wolverhampton Wanderers,Tottenham Hotspur,2,3,A,3.10,3.40,2.45,0.314755,...,0.398262,A,1,0,0,1,Wolverhampton Wanderers2019,Tottenham Hotspur2019,178.02,751.05
3418,2019,Wolverhampton Wanderers,Watford,0,2,A,1.75,3.75,5.25,0.555556,...,0.185185,H,0,0,0,1,Wolverhampton Wanderers2019,Watford2019,178.02,148.68


For our regression we want to use the logarithm of the ratio of hTM to aTM, so we create this variable next:

In [16]:
gbygdat['lhTMratio'] = np.log(gbygdat['hTM']/gbygdat['aTM'])
gbygdat

,seasonyrend,home_team,away_team,FTHG,FTAG,FTR,B365H,B365D,B365A,B365HPr,...,B365res,B365true,Houtcome,Doutcome,Aoutcome,hteamid,ateamid,hTM,aTM,lhTMratio
0,2011,Aston Villa,Arsenal,2,4,A,3.40,3.30,2.20,0.279661,...,A,1,0,0,1,Aston Villa2011,Arsenal2011,127.94,272.70,-0.756811
1,2011,Birmingham City,Arsenal,0,3,A,5.50,3.60,1.67,0.171786,...,A,1,0,0,1,Birmingham City2011,Arsenal2011,68.00,272.70,-1.388865
2,2011,Blackburn Rovers,Arsenal,1,2,A,6.00,4.00,1.57,0.158186,...,A,1,0,0,1,Blackburn Rovers2011,Arsenal2011,65.93,272.70,-1.419779
3,2011,Blackpool,Arsenal,1,3,A,8.50,5.00,1.36,0.111732,...,A,1,0,0,1,Blackpool2011,Arsenal2011,24.37,272.70,-2.415019
4,2011,Bolton Wanderers,Arsenal,2,1,H,4.50,3.80,1.75,0.210277,...,A,0,1,0,0,Bolton Wanderers2011,Arsenal2011,74.93,272.70,-1.291818
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3415,2019,Wolverhampton Wanderers,Newcastle United,1,1,D,1.70,3.75,5.75,0.571760,...,H,0,0,1,0,Wolverhampton Wanderers2019,Newcastle United2019,178.02,159.03,0.112803
3416,2019,Wolverhampton Wanderers,Southampton,2,0,H,1.80,3.60,5.25,0.542636,...,H,1,1,0,0,Wolverhampton Wanderers2019,Southampton2019,178.02,239.72,-0.297576
3417,2019,Wolverhampton Wanderers,Tottenham Hotspur,2,3,A,3.10,3.40,2.45,0.314755,...,A,1,0,0,1,Wolverhampton Wanderers2019,Tottenham Hotspur2019,178.02,751.05,-1.439576
3418,2019,Wolverhampton Wanderers,Watford,0,2,A,1.75,3.75,5.25,0.555556,...,H,0,0,0,1,Wolverhampton Wanderers2019,Watford2019,178.02,148.68,0.180100


Finally, in order to run the regression, the package we need to use requires the ordered variable (H, D, A) to be given a numeric value. We can do this by defining a new variable, win value, which equal 2 if H, 1 if D and 0 if A. This is what the next line of code does:

In [17]:
gbygdat['winvalue'] = np.where(gbygdat['FTR'] == 'H', 2 ,(np.where(gbygdat['FTR'] == 'D', 1, 0)))
gbygdat

,seasonyrend,home_team,away_team,FTHG,FTAG,FTR,B365H,B365D,B365A,B365HPr,...,B365true,Houtcome,Doutcome,Aoutcome,hteamid,ateamid,hTM,aTM,lhTMratio,winvalue
0,2011,Aston Villa,Arsenal,2,4,A,3.40,3.30,2.20,0.279661,...,1,0,0,1,Aston Villa2011,Arsenal2011,127.94,272.70,-0.756811,0
1,2011,Birmingham City,Arsenal,0,3,A,5.50,3.60,1.67,0.171786,...,1,0,0,1,Birmingham City2011,Arsenal2011,68.00,272.70,-1.388865,0
2,2011,Blackburn Rovers,Arsenal,1,2,A,6.00,4.00,1.57,0.158186,...,1,0,0,1,Blackburn Rovers2011,Arsenal2011,65.93,272.70,-1.419779,0
3,2011,Blackpool,Arsenal,1,3,A,8.50,5.00,1.36,0.111732,...,1,0,0,1,Blackpool2011,Arsenal2011,24.37,272.70,-2.415019,0
4,2011,Bolton Wanderers,Arsenal,2,1,H,4.50,3.80,1.75,0.210277,...,0,1,0,0,Bolton Wanderers2011,Arsenal2011,74.93,272.70,-1.291818,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3415,2019,Wolverhampton Wanderers,Newcastle United,1,1,D,1.70,3.75,5.75,0.571760,...,0,0,1,0,Wolverhampton Wanderers2019,Newcastle United2019,178.02,159.03,0.112803,1
3416,2019,Wolverhampton Wanderers,Southampton,2,0,H,1.80,3.60,5.25,0.542636,...,1,1,0,0,Wolverhampton Wanderers2019,Southampton2019,178.02,239.72,-0.297576,2
3417,2019,Wolverhampton Wanderers,Tottenham Hotspur,2,3,A,3.10,3.40,2.45,0.314755,...,1,0,0,1,Wolverhampton Wanderers2019,Tottenham Hotspur2019,178.02,751.05,-1.439576,0
3418,2019,Wolverhampton Wanderers,Watford,0,2,A,1.75,3.75,5.25,0.555556,...,0,0,0,1,Wolverhampton Wanderers2019,Watford2019,178.02,148.68,0.180100,0


Our df has a lot of variables, many of which we do not need, so for the sake of tidying up, we now drop some variables. Note that the ones we drop could easily be recreated using the variables that remain.

In [18]:
gbygdat= gbygdat.drop(columns= ['seasonyrend', 'home_team', 'away_team','B365H','B365D','B365A'])
gbygdat

,FTHG,FTAG,FTR,B365HPr,B365DPr,B365APr,B365res,B365true,Houtcome,Doutcome,Aoutcome,hteamid,ateamid,hTM,aTM,lhTMratio,winvalue
0,2,4,A,0.279661,0.288136,0.432203,A,1,0,0,1,Aston Villa2011,Arsenal2011,127.94,272.70,-0.756811,0
1,0,3,A,0.171786,0.262451,0.565763,A,1,0,0,1,Birmingham City2011,Arsenal2011,68.00,272.70,-1.388865,0
2,1,2,A,0.158186,0.237280,0.604534,A,1,0,0,1,Blackburn Rovers2011,Arsenal2011,65.93,272.70,-1.419779,0
3,1,3,A,0.111732,0.189944,0.698324,A,1,0,0,1,Blackpool2011,Arsenal2011,24.37,272.70,-2.415019,0
4,2,1,H,0.210277,0.249012,0.540711,A,0,1,0,0,Bolton Wanderers2011,Arsenal2011,74.93,272.70,-1.291818,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3415,1,1,D,0.571760,0.259198,0.169042,H,0,0,1,0,Wolverhampton Wanderers2019,Newcastle United2019,178.02,159.03,0.112803,1
3416,2,0,H,0.542636,0.271318,0.186047,H,1,1,0,0,Wolverhampton Wanderers2019,Southampton2019,178.02,239.72,-0.297576,2
3417,2,3,A,0.314755,0.286983,0.398262,A,1,0,0,1,Wolverhampton Wanderers2019,Tottenham Hotspur2019,178.02,751.05,-1.439576,0
3418,0,2,A,0.555556,0.259259,0.185185,H,0,0,0,1,Wolverhampton Wanderers2019,Watford2019,178.02,148.68,0.180100,0


### Estimation

Now we've organized the data we're ready to run the regressions.

In week 1 we encountered the ordered logistic regression, which enables us to estimate the relationship between a dependent variable and independent variable, where the dependent variable consists of a fixed number of discrete possibilities. Here these possibilities are win, draw or lose. 

From the logistic regression, we can identify the probability associated with each possible outcome, as a function of our independent variables. In our case, we will estimate that probability for each of the three possible results for every game in our dataset, depending on the ratio of the TM value of the home team to the TM value of the away team (lhTMratio). 

First, we install the package we need to run the regression:

In [19]:
# This is the package we need to run the ordered logit

from bevel.linear_ordinal_regression import OrderedLogit
ol = OrderedLogit()

The regression command is ol.fit, and we specify first the explanatory variable (lhTMratio) and the dependent variable (winvalue).

Unlike the linear regression model which also gives us a constant for the regression, the ordered logit gives us estimates of thresholds between ordered groups (2, 1 and 0, signifying H, D, A). These estimates can then be used to generate estimates of H, D, A assuming that the explanatory variable lhTMratio has a value of zero. In that respect, these estimates are like constants for each possible outcome.

The regression output obtained when you use ol.print_summary() is a little messy, but it gives us the estimate of the coefficient (beta) of lhTMratio, the standard error of the estimate (SE(beta)), the pvalue, and the upper and lower confidence intervals:

In [20]:
ol.fit(gbygdat['lhTMratio'], gbygdat['winvalue'])
ol.print_summary()

n=3420
                  beta  se(beta)      p  lower 0.95  upper 0.95     
attribute names                                                     
column_1        0.7595    0.0343 0.0000      0.6923      0.8267  ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1 

Somers' D = 0.301


We can see the value of the coefficient of lhTMratio is 0.7595 and the standard error is 0.0343, yielding a t-statistic of over 16, implying that the coefficient is statistically significant at the 0.01 level and better. The higher the ratio, the better the outcome, as viewed by the home team. 

We want to convert our estimates into probabilities, and to do this we need the estimates of the intercepts as well. the coefficient for lhTMratio is stored as ol.coef_[0], while the threshold between A and D is stored as ol.coef_[1] The threshold between D and H is stored as ol.coef_[2] We can print these out with appropriate names:

In [21]:
print(f'beta = {ol.coef_[0]:.4f}')
print(f'intercept_AD = {ol.coef_[1]:.4f}')
print(f'intercept_DH = {ol.coef_[2]:.4f}')

beta = 0.7595
intercept_AD = -0.9868
intercept_DH = 0.2002


In [22]:
# If needed, we can also obtain the standard errors (in order for coefficients 0, 1 and 2):

print(ol.se_)

[0.03427444 0.04021741 0.0365153 ]


To generate the predicted probabilities we need to manipulate the coefficients. The logit regression equation has the form log(p/(1-p)) = a + bX. Rearranging this equation we can obtain p = 1/(1+ exp(a +bX)).

For each game, we know X (lhTMratio) and now we know b (0.7595). Since this is an ordered logit with three possible outcomes, the probability of the worst outcome A (viewed by the home team) depends on intercept_AD (ol.coef_[1]), the probability of the middle outcome D depends on intercept_AD and intercept_DH (ol.coef_[1] and ol.coef_[2]), while the probability of the best outcome depends on intercept_DH.

If we calculate the probability of A first, using ol.coef_[0] and ol.coef_[1], then when calculating the probability of D we can use the fact that we already have the probability of A, and also when calculating the probability of H we can use the fact that we already have the probability of D. 

Thus we now create the predicted values of the H, A and D probabilities from our model:

In [23]:
gbygdat['predA'] = 1/(1+np.exp(-(ol.coef_[1]-ol.coef_[0]*gbygdat['lhTMratio'])))
gbygdat['predD'] = 1/(1+np.exp(-(ol.coef_[2]-ol.coef_[0]*gbygdat['lhTMratio']))) - gbygdat['predA']
gbygdat['predH'] = 1 - gbygdat['predA'] - gbygdat['predD']

gbygdat

,FTHG,FTAG,FTR,B365HPr,B365DPr,B365APr,B365res,B365true,Houtcome,Doutcome,Aoutcome,hteamid,ateamid,hTM,aTM,lhTMratio,winvalue,predA,predD,predH
0,2,4,A,0.279661,0.288136,0.432203,A,1,0,0,1,Aston Villa2011,Arsenal2011,127.94,272.70,-0.756811,0,0.398435,0.286160,0.315405
1,0,3,A,0.171786,0.262451,0.565763,A,1,0,0,1,Birmingham City2011,Arsenal2011,68.00,272.70,-1.388865,0,0.517005,0.261160,0.221835
2,1,2,A,0.158186,0.237280,0.604534,A,1,0,0,1,Blackburn Rovers2011,Arsenal2011,65.93,272.70,-1.419779,0,0.522865,0.259326,0.217809
3,1,3,A,0.111732,0.189944,0.698324,A,1,0,0,1,Blackpool2011,Arsenal2011,24.37,272.70,-2.415019,0,0.700020,0.184337,0.115643
4,2,1,H,0.210277,0.249012,0.540711,A,0,1,0,0,Bolton Wanderers2011,Arsenal2011,74.93,272.70,-1.291818,2,0.498585,0.266596,0.234819
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3415,1,1,D,0.571760,0.259198,0.169042,H,0,0,1,0,Wolverhampton Wanderers2019,Newcastle United2019,178.02,159.03,0.112803,1,0.254938,0.273659,0.471403
3416,2,0,H,0.542636,0.271318,0.186047,H,1,1,0,0,Wolverhampton Wanderers2019,Southampton2019,178.02,239.72,-0.297576,2,0.318479,0.286484,0.395037
3417,2,3,A,0.314755,0.286983,0.398262,A,1,0,0,1,Wolverhampton Wanderers2019,Tottenham Hotspur2019,178.02,751.05,-1.439576,0,0.526615,0.258127,0.215258
3418,0,2,A,0.555556,0.259259,0.185185,H,0,0,0,1,Wolverhampton Wanderers2019,Watford2019,178.02,148.68,0.180100,0,0.245352,0.270493,0.484155


From these probabilities, we now want to identify both the most likely outcome prediction and the Brier Score.

To identify the most likely outcome, we first identify the largest value in the three columns, predA, predD and predH, which we call Maxprob, and then we create a value for the model's prediction (logitpred) where for the value of Maxprob.

To enable us to see all the columns, we set max_columns using pd.set_option equal to 30.

In [24]:
gbygdat['Maxprob'] =gbygdat[['predA','predD','predH']].max(axis=1)
pd.set_option('display.max_columns', 30)
gbygdat

,FTHG,FTAG,FTR,B365HPr,B365DPr,B365APr,B365res,B365true,Houtcome,Doutcome,Aoutcome,hteamid,ateamid,hTM,aTM,lhTMratio,winvalue,predA,predD,predH,Maxprob
0,2,4,A,0.279661,0.288136,0.432203,A,1,0,0,1,Aston Villa2011,Arsenal2011,127.94,272.70,-0.756811,0,0.398435,0.286160,0.315405,0.398435
1,0,3,A,0.171786,0.262451,0.565763,A,1,0,0,1,Birmingham City2011,Arsenal2011,68.00,272.70,-1.388865,0,0.517005,0.261160,0.221835,0.517005
2,1,2,A,0.158186,0.237280,0.604534,A,1,0,0,1,Blackburn Rovers2011,Arsenal2011,65.93,272.70,-1.419779,0,0.522865,0.259326,0.217809,0.522865
3,1,3,A,0.111732,0.189944,0.698324,A,1,0,0,1,Blackpool2011,Arsenal2011,24.37,272.70,-2.415019,0,0.700020,0.184337,0.115643,0.700020
4,2,1,H,0.210277,0.249012,0.540711,A,0,1,0,0,Bolton Wanderers2011,Arsenal2011,74.93,272.70,-1.291818,2,0.498585,0.266596,0.234819,0.498585
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3415,1,1,D,0.571760,0.259198,0.169042,H,0,0,1,0,Wolverhampton Wanderers2019,Newcastle United2019,178.02,159.03,0.112803,1,0.254938,0.273659,0.471403,0.471403
3416,2,0,H,0.542636,0.271318,0.186047,H,1,1,0,0,Wolverhampton Wanderers2019,Southampton2019,178.02,239.72,-0.297576,2,0.318479,0.286484,0.395037,0.395037
3417,2,3,A,0.314755,0.286983,0.398262,A,1,0,0,1,Wolverhampton Wanderers2019,Tottenham Hotspur2019,178.02,751.05,-1.439576,0,0.526615,0.258127,0.215258,0.526615
3418,0,2,A,0.555556,0.259259,0.185185,H,0,0,0,1,Wolverhampton Wanderers2019,Watford2019,178.02,148.68,0.180100,0,0.245352,0.270493,0.484155,0.484155


In [25]:
# the predicted result

gbygdat['logitpred']=np.where(gbygdat['Maxprob']==gbygdat['predA'],'A',np.where(gbygdat['Maxprob']==gbygdat['predD'],'D','H'))
gbygdat

,FTHG,FTAG,FTR,B365HPr,B365DPr,B365APr,B365res,B365true,Houtcome,Doutcome,Aoutcome,hteamid,ateamid,hTM,aTM,lhTMratio,winvalue,predA,predD,predH,Maxprob,logitpred
0,2,4,A,0.279661,0.288136,0.432203,A,1,0,0,1,Aston Villa2011,Arsenal2011,127.94,272.70,-0.756811,0,0.398435,0.286160,0.315405,0.398435,A
1,0,3,A,0.171786,0.262451,0.565763,A,1,0,0,1,Birmingham City2011,Arsenal2011,68.00,272.70,-1.388865,0,0.517005,0.261160,0.221835,0.517005,A
2,1,2,A,0.158186,0.237280,0.604534,A,1,0,0,1,Blackburn Rovers2011,Arsenal2011,65.93,272.70,-1.419779,0,0.522865,0.259326,0.217809,0.522865,A
3,1,3,A,0.111732,0.189944,0.698324,A,1,0,0,1,Blackpool2011,Arsenal2011,24.37,272.70,-2.415019,0,0.700020,0.184337,0.115643,0.700020,A
4,2,1,H,0.210277,0.249012,0.540711,A,0,1,0,0,Bolton Wanderers2011,Arsenal2011,74.93,272.70,-1.291818,2,0.498585,0.266596,0.234819,0.498585,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3415,1,1,D,0.571760,0.259198,0.169042,H,0,0,1,0,Wolverhampton Wanderers2019,Newcastle United2019,178.02,159.03,0.112803,1,0.254938,0.273659,0.471403,0.471403,H
3416,2,0,H,0.542636,0.271318,0.186047,H,1,1,0,0,Wolverhampton Wanderers2019,Southampton2019,178.02,239.72,-0.297576,2,0.318479,0.286484,0.395037,0.395037,H
3417,2,3,A,0.314755,0.286983,0.398262,A,1,0,0,1,Wolverhampton Wanderers2019,Tottenham Hotspur2019,178.02,751.05,-1.439576,0,0.526615,0.258127,0.215258,0.526615,A
3418,0,2,A,0.555556,0.259259,0.185185,H,0,0,0,1,Wolverhampton Wanderers2019,Watford2019,178.02,148.68,0.180100,0,0.245352,0.270493,0.484155,0.484155,H


Now we have the prediction for the most likely outcome, we can identify cases where the prediction was correct, and where it was not, just as we did for the betting odds:

In [26]:
gbygdat['logittrue']= np.where(gbygdat['logitpred'] == gbygdat['FTR'],1,0)
gbygdat

,FTHG,FTAG,FTR,B365HPr,B365DPr,B365APr,B365res,B365true,Houtcome,Doutcome,Aoutcome,hteamid,ateamid,hTM,aTM,lhTMratio,winvalue,predA,predD,predH,Maxprob,logitpred,logittrue
0,2,4,A,0.279661,0.288136,0.432203,A,1,0,0,1,Aston Villa2011,Arsenal2011,127.94,272.70,-0.756811,0,0.398435,0.286160,0.315405,0.398435,A,1
1,0,3,A,0.171786,0.262451,0.565763,A,1,0,0,1,Birmingham City2011,Arsenal2011,68.00,272.70,-1.388865,0,0.517005,0.261160,0.221835,0.517005,A,1
2,1,2,A,0.158186,0.237280,0.604534,A,1,0,0,1,Blackburn Rovers2011,Arsenal2011,65.93,272.70,-1.419779,0,0.522865,0.259326,0.217809,0.522865,A,1
3,1,3,A,0.111732,0.189944,0.698324,A,1,0,0,1,Blackpool2011,Arsenal2011,24.37,272.70,-2.415019,0,0.700020,0.184337,0.115643,0.700020,A,1
4,2,1,H,0.210277,0.249012,0.540711,A,0,1,0,0,Bolton Wanderers2011,Arsenal2011,74.93,272.70,-1.291818,2,0.498585,0.266596,0.234819,0.498585,A,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3415,1,1,D,0.571760,0.259198,0.169042,H,0,0,1,0,Wolverhampton Wanderers2019,Newcastle United2019,178.02,159.03,0.112803,1,0.254938,0.273659,0.471403,0.471403,H,0
3416,2,0,H,0.542636,0.271318,0.186047,H,1,1,0,0,Wolverhampton Wanderers2019,Southampton2019,178.02,239.72,-0.297576,2,0.318479,0.286484,0.395037,0.395037,H,1
3417,2,3,A,0.314755,0.286983,0.398262,A,1,0,0,1,Wolverhampton Wanderers2019,Tottenham Hotspur2019,178.02,751.05,-1.439576,0,0.526615,0.258127,0.215258,0.526615,A,1
3418,0,2,A,0.555556,0.259259,0.185185,H,0,0,0,1,Wolverhampton Wanderers2019,Watford2019,178.02,148.68,0.180100,0,0.245352,0.270493,0.484155,0.484155,H,0


As we did with the betting odds, we can identify the percentage of cases where the result was correctly identified by our model:

gbygdat['logittrue'].mean()

And finally, we can create a Brier Score for our model, just like we did with the betting odds.

In [27]:
Brierlogit = ((gbygdat['predH'] - gbygdat['Houtcome'])**2 +(gbygdat['predD'] - gbygdat['Doutcome'])**2 +\
             (gbygdat['predA'] - gbygdat['Aoutcome'])**2).sum()/3420 
Brierlogit

0.5842199930783991

## Step 3: Comparison

**Predicted outcomes** - the bookmaker odds were correct 54% of the time, while our model was correct 50% of the time. 

**Brier Score** - the Brier score for bookmaker odds was 0.569 whereas the Brier Score for our model was 0.611. Recall that a higher Brier score means a larger deviation from the actual result, and therefore a worse performance, not a better one.

These values are clearly very close. As we should have expected, our model does not perform quite as well as the bookmaker model. However, the bookmakers process all available information when setting the odds- the form of the players, the team, injuries, the weather, the performance of the coach, and all the gossip from the media. Our model relied on just two pieces of information- the TM value ratio and the identity of the home team. Given this limited information, the performance of the model actually seems quite remarkably good.

Comparing Brier Score is not immediately intuitive - is .611 close to .569? Unlike a percentage, the basis of comparison is not obvious. However, here's one simple comparison. If you took the bookmakers odds, and then for each game took 2% of the probability of a home team win, and added 1% each to the probability of a draw or an away team win, this would create a larger gap in the Brier Scores than we found between the bookmaker odds and our model. 

One thing that may have already struck you is that if the performance of the bookmakers and our model are so similar, perhaps they are generating the same predictions. We can calculate what percentage of the predicted outcomes are the same using np.where:

In [28]:
#same prediction

samepercent = np.where(gbygdat['B365res'] == gbygdat['logitpred'],1,0).sum()/3240
samepercent

0.9317901234567901

With 85% of the predictions being the same, it seems reasonable to conclude our model is capturing most of the variability that we see in the bookmaking market. Thus we might argue that the TM value ratio *explains* the process which determines setting bookmaking odds. However, in constructing the model we used the outcomes of games as inputs for the model which we then used to 'predict' the outcomes of those same games, which all seems a little circular.  We now want to use the model to *forecast* the outcome of games before they have taken place. In other words to move from within sample prediction to out-of-sample prediction.